# Audio estéreo

In [ ]:

# Importacion.
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt

# Directorios que usaremos.
cwd = os.getcwd()
audio_input_path = os.path.join(cwd, os.path.join('audio', '_input'))  # audio_examples
audio_output_path = os.path.join(cwd, os.path.join('audio', '_output'))
print(f'Path to input audio: {audio_input_path}')
print(f'Path to output audio: {audio_output_path}\n')


# Cargamos el archivo de audio.
filename = os.path.join(audio_input_path, 'breaking_bad.wav')
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')



# Mostrar informacion (sonido estéreo).
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}')
print(f'- 1º canal:   {audio_data[:5, 0]}...')
print(f'- 2º canal:   {audio_data[:5, 1]}...')
print(f'- Resolucion: {type(audio_data[0,0])}\n')



In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate) # .T se pasa únicamente si es audio estéreo.


# Audio mono

In [ ]:

# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Column-wise.
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')
print(f'- Canal unico:  {new_data_mono[:5]}...')

# Mantenemos la misma resolucion que antes.
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')


In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, 'sample1_mono.wav'),
    rate=sample_rate,
    data=new_data_mono
)

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

In [ ]:
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz\n')

## Comparación del tipo de audios

In [ ]:
!ls -sh audio/_input/breaking_brad.wav
!ls -sh audio/_output/sample1_mono.wav

# Gráfica del dominio del tiempo

In [ ]:
ampl_values_estereo = len(audio_data)
ampl_values_mono = len(new_data_mono)
print(f'Número de muestras del audio (valores de amplitud): {ampl_values_estereo}')
print(f'Número de muestras del audio (valores de amplitud): {ampl_values_mono}')

In [ ]:
# Construimos el array para el eje x que representa el tiempo de la grabación.
# Tiene la forma: np.arange(Vi, Vf, P). Explicado a continuación.
t1 = np.arange(0, ampl_values_estereo/sample_rate, 1/sample_rate)
t2 = np.arange(0, ampl_values_mono/sample_rate, 1/sample_rate)

In [ ]:
print(t1)
print(t2)

In [ ]:
# Creamos la figura.
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Solo mostramos las primeras 50 muestras de amplitud (por claridad).
begin = 7500
end = 10000

# Señal a 48 kHz.
ax[0].plot(t1[begin:end], audio_data[begin:end], marker='X')
ax[0].set_title(f'Audio en el dominio del tiempo muestreado del audio estéreo')
ax[0].set_ylabel('Amplitud')
ax[0].grid(True)

ax[1].plot(t2[begin:end], new_data_mono[begin:end], c='tab:red', marker='X')
ax[1].set_title(f'Audio en el dominio del tiempo muestreado del audio mono')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Amplitud')
ax[1].grid(True)

# Mostramos la figura.
plt.tight_layout()
plt.show()

Se ha añadido al inicio del código "begin = 7500" debido a que el audio de breaking_bad tarda en empezar y así se puede observar perfectamente la gráfica del dominio del tiempo de los audios estéreo y mono.
Se ha quitado la variable ratio, ya que se utilizaba en el notebook proporcionado por el profesor porque los audios eran de diferente frecuencia.

## Audio estéreo y mono
En el audio mono se escucha el mismo sonido por el auricular derecho que por el auricular izquierdo, mientras que en el estéreo se escuchan sonidos diferentes aunque en su combinación reproduzcan el mismo sonido.
## ¿Qué es la frecuencia de muestreo?
Es el número de muestras por segundo que tomamos del audio.

## ¿Qué es el aliasing?

Es un efecto que provoca que señales continuas las cuales sean diferentes, se muestren iguales, debido a una frecuencia de muestreo demasiado baja.

## ¿Qué es la profundidad de bits?

Indica cuantos bits hay disponibles para medir la onda sonora, además para que se almacenen nuestras muestras en bytes digitales.

## ¿Qué es la tasa de bits?

Indica el tamaño del archivo de audio digital

# Transformada de Fourirer (FFT)

In [ ]:
# La longitud del array de datos y el
# sample rate (frecuencia de muestreo).
n = len(new_data_mono)
Fs = sample_rate

# Working with stereo audio, there are two channels in the audio data.
# Let's retrieve each channel seperately:
# ch1 = np.array([data[i][0] for i in range(n)]) #channel 1
# ch2 = np.array([data[i][1] for i in range(n)]) #channel 2
# We can then perform a Fourier analysis on the first
# channel to see what the spectrum looks like.

# Calculando la Transformada Rapida de Fourier (FFT) en audio mono.
ch_Fourier = np.fft.fft(new_data_mono)  # ch1

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

Se realiza con el audio mono y se puede observar que hay solo un canal, se carga el audio mono utilizando "new_data_mono" la variable sample_rate es igual tanto en estéreo como en mono.

## ¿Por qué se realiza la Transformada de Fourier?
Se utiliza para descomponer la señal del audio en sus componentes espectrales individuales, aportando información de su composición.

# Energía del espectograma y frecuencia de corte

In [ ]:
# Definimos epsilon: la parte de la energia
# del espectro que no conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (CAMBIAD ESTO).
eps = eps[6]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el valor
# de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

# Compresión de la onda

In [ ]:
# Definimos los nombres de los audios comprimidos.
wav_compressed_file = "sample_48kHz_compressed.wav"

# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = new_data_mono[::D]

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(audio_output_path, wav_compressed_file))

En el código de este apartado y el anterior, contamos con un vector con 7 valores distintos para epsilon, cuanto mayor sea el valor de epsilon, mayor energía va a descartar en la pista del audio, entonces se escuchará una versión empeorada del audio mono, si la constante epsilon es muy pequeña la energía que se descarta es mínima por lo cual se esuchará el audio apenas sin diferencias con el audio mono original.
También se puede observar el valor de corte para la energía, que es el valor de la energía en el momento del corte.
La máscara indica true si epsilon tiene un valor que pertenece a un fragmento eliminado o false si epsilon contiene un valor que no se debe eliminar.
La frecuencia de corte que es cuando a partir de esa frecuencia se descarta la energía.


# Mostrar el espectograma de ambas ondas: original y comprimida

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(new_data_mono, NFFT=1024, Fs=sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
!ls -sh audio/_output/sample_48kHz_compressed.wav

In [ ]:
!ls -sh audio/_output/sample1_mono.wav

Se puede observar como el audio comprimido ocupa mucho menos espacio

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)